In [15]:
import random
import numpy as np 
import matplotlib.pyplot as plt 
import time 

# Générer le graphe TSP incomplet
def generate_tsp_incomplete(num_sommets, max_poids_arete):
    matrice = [[None for _ in range(num_sommets)] for _ in range(num_sommets)]
    intervalles = {}
    for i in range(num_sommets):
        intervalle_start = random.randint(0, 10)
        intervalle_end = intervalle_start + random.randint(5, 20)
        intervalles[i] = (intervalle_start, intervalle_end)
    for i in range(num_sommets):
        for j in range(i + 1, num_sommets):
            if random.choice([True, False]):
                poids = random.randint(1, max_poids_arete)
                matrice[i][j] = poids
                matrice[j][i] = poids
    for i in range(num_sommets):
        pas_connexe = True
        for j in range(num_sommets):
            if matrice[i][j] is None:
                matrice[i][j] = float('inf')
            else:
                pas_connexe = False
        if pas_connexe:
            poids = random.randint(1, max_poids_arete)
            matrice[i][num_sommets-1] = poids
            matrice[num_sommets-1][i] = poids
    return np.array(matrice), intervalles

num_sommets = 5
max_poids_arete = 10 

# Générer les données du problème
tsp_matrice, tsp_intervalles = generate_tsp_incomplete(num_sommets, max_poids_arete)

# Afficher la matrice pondérée
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)

# Afficher les intervalles
print("\nIntervalles des sommets")
print(tsp_intervalles)  


Matrice pondérée:
[inf inf inf inf  4.]
[inf inf  9.  9.  6.]
[inf  9. inf  7.  9.]
[inf  9.  7. inf 10.]
[ 4.  6.  9. 10. inf]

Intervalles des sommets
{0: (4, 9), 1: (7, 22), 2: (4, 16), 3: (4, 13), 4: (8, 17)}


In [16]:
import random
import numpy as np 

def is_valid_solution(solution, matrice, intervalles):
    # Validate if the solution respects the interval constraints
    path_length = calculate_path_length(matrice, solution)
    for city in range(len(solution)):
        min_weight, max_weight = intervalles[city]
        if not (min_weight <= path_length <= max_weight):
            return False
    return True

def calculate_path_length(matrice, solution):
    # Calculate the total length of the path
    length = 0
    for i in range(len(solution) - 1):
        length += matrice[solution[i]][solution[i + 1]]
    length += matrice[solution[-1]][solution[0]]  # return to the start point
    return length

def generate_neighbor_solution(current_solution, matrice):
    # Generate a neighboring solution that respects the interval constraints
    while True:
        neighbor = current_solution[:]
        i, j = random.sample(range(len(neighbor)), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        
        if is_valid_solution(neighbor, matrice):
            return neighbor 

# Implémentation de l'algorithme hill climbing
def hill_climbing(matrice, num_iterations=1000):
    num_sommets = len(matrice)
    execution_times = []
    distances = []
    while True:
        start_time = time.time()
        current_solution = list(range(num_sommets))
        random.shuffle(current_solution)
        if is_valid_solution(current_solution, matrice):
            break
    current_length = calculate_path_length(matrice, current_solution)
    
    for _ in range(num_iterations):
        # Generate a valid neighboring solution
        next_solution = generate_neighbor_solution(current_solution, matrice) 
        next_length = calculate_path_length(matrice, next_solution)
        
        # If the neighboring solution is better, adopt it
        if next_length < current_length:
            current_solution = next_solution
            current_length = next_length
        
        end_time = time.time()
        execution_times.append(end_time - start_time)
        distances.append(current_length)  # Record the distance, not the solution
    
    return current_solution, current_length, execution_times, distances 

In [17]:
def plot_route(path, points):
    plt.figure(figsize=(10, 5))
    plt.scatter(points[:, 0], points[:, 1], c='blue')
    for i in range(len(path) - 1):
        from_node = path[i]
        to_node = path[i + 1]
        plt.plot([points[from_node, 0], points[to_node, 0]], [points[from_node, 1], points[to_node, 1]], 'k-')
    plt.title('Best Route Found')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.show()    

In [18]:
# best_path, best_path_length = hill_climbing()   
# print(f"Meilleur chemin trouvé pour eme iteration : {best_path} avec une longueur de {best_path_length}")   

In [19]:
tsp_matrice, tsp_intervalles = generate_tsp_incomplete(num_sommets, max_poids_arete)

# Afficher la matrice pondérée
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)

# Afficher les intervalles
print("\nIntervalles des sommets:")
print(tsp_intervalles)

# Résoudre le problème avec l'algorithme hill climbing
solution, length = hill_climbing(tsp_matrice)

# Afficher la solution trouvée et sa longueur
print("\nSolution trouvée par hill climbing:")
print("Chemin:", solution)
print("Longueur du chemin:", length)  


Matrice pondérée:
[inf 10.  9. inf inf]
[10. inf inf inf  7.]
[ 9. inf inf  2. inf]
[inf inf  2. inf  1.]
[inf  7. inf  1. inf]

Intervalles des sommets:
{0: (7, 13), 1: (10, 24), 2: (5, 22), 3: (7, 24), 4: (2, 7)}


TypeError: is_valid_solution() missing 1 required positional argument: 'intervalles'